## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Saint-Philippe,-21.3585,55.7679,71.19,74,65,13.98,RE,light rain
1,1,Aswan,24.0934,32.9070,85.39,13,0,10.25,EG,clear sky
2,2,Durres,41.3231,19.4414,62.78,78,1,5.37,AL,clear sky
3,3,Busselton,-33.6500,115.3333,56.17,82,57,5.26,AU,broken clouds
4,4,Kodiak,57.7900,-152.4072,54.91,71,0,6.91,US,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?65
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Saint-Philippe,-21.3585,55.7679,71.19,74,65,13.98,RE,light rain
1,1,Aswan,24.0934,32.9070,85.39,13,0,10.25,EG,clear sky
6,6,Beterou,9.1992,2.2586,74.53,88,98,1.81,BJ,light rain
7,7,San Patricio,28.0170,-97.5169,86.54,61,1,22.30,US,clear sky
12,12,Saraland,30.8207,-88.0706,82.42,77,71,8.21,US,broken clouds
14,14,Mayo,38.8876,-76.5119,78.96,71,0,5.75,US,clear sky
15,15,Hithadhoo,-0.6000,73.0833,83.30,69,99,10.11,MV,overcast clouds
16,16,Albany,42.6001,-73.9662,70.43,60,22,2.62,US,few clouds
17,17,Fairbanks,64.8378,-147.7164,73.60,30,0,13.80,US,clear sky
19,19,Souillac,-20.5167,57.5167,73.78,56,75,17.27,MU,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                361
City                   361
Lat                    361
Lng                    361
Max Temp               361
Humidity               361
Cloudiness             361
Wind Speed             361
Country                361
Current Description    361
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Saint-Philippe,RE,71.19,light rain,-21.3585,55.7679,
1,Aswan,EG,85.39,clear sky,24.0934,32.9070,
6,Beterou,BJ,74.53,light rain,9.1992,2.2586,
7,San Patricio,US,86.54,clear sky,28.0170,-97.5169,
12,Saraland,US,82.42,broken clouds,30.8207,-88.0706,
14,Mayo,US,78.96,clear sky,38.8876,-76.5119,
15,Hithadhoo,MV,83.30,overcast clouds,-0.6000,73.0833,
16,Albany,US,70.43,few clouds,42.6001,-73.9662,
17,Fairbanks,US,73.60,clear sky,64.8378,-147.7164,
19,Souillac,MU,73.78,light rain,-20.5167,57.5167,


In [39]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [40]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"] = hotel_df["Hotel Name"].map(lambda x: x.strip())
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != "")]
clean_hotel_df.head()
#len(hotel_df["Hotel Name"].head(0))
#hotel_df.dropna()
#clean_hotel_df = hotel_df.copy().hotel_df.dropna(subset=["Hotel Name"], inplace=True)
#hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Saint-Philippe,RE,71.19,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
1,Aswan,EG,85.39,clear sky,24.0934,32.9070,Nuba Nile
6,Beterou,BJ,74.53,light rain,9.1992,2.2586,Monastère de Kokoubou
7,San Patricio,US,86.54,clear sky,28.0170,-97.5169,"Motel 6 Sinton, TX"
12,Saraland,US,82.42,broken clouds,30.8207,-88.0706,Microtel Inn & Suites by Wyndham Saraland/Nort...


In [41]:
# 8a. Create the output File (CSV)
output_data_file = "weather_Data/clean_hotel_df.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [42]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [44]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))